# Request: PWA (Crashes) 


#### 1. Database Connections for DOTDEVRHPGSQL01, DOT55SQL01
####     Tables: CRASHDATA, FORMS
#### 2. 
#### 3.

In [12]:
from ris import db2  #library designed for SQL database connection and querying
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
print 'Notebook run: {}'.format(timestamp)
print os.getcwd()
# %load_ext sql 

Notebook run: 2019-08-15 11:36
E:\RIS\Staff Folders\Samuel\Requests\SH\PWA


### Database connections

In [6]:
cdb = db2.PostgresDb('DOTDEVRHPGSQL01', 'CRASHDATA', quiet = True)
fdb = db2.SqlDb('DOT55SQL01', 'forms', user='arcgis', db_pass='arcgis', quiet = True)

User name (CRASHDATA):SOGE
Password (CRASHDATA)········


In [13]:
def all_crashes():
    
    start = (datetime.datetime.now() - relativedelta(years=3)).strftime('%Y-%m-%d')
    end = datetime.datetime.now().strftime('%Y-%m-%d') 

    crashes = db2.query_to_table(fdb,"""
    
        select coalesce(c.nodeid, c.lion_node_number) nodeid, 
        integration_id, cast(c.accident_dt as date) accident_dt
        from forms.dbo.wc_accident_f c
        where year(c.accident_dt) >= 2017
        and accident_dt between '{start}' and '{end}'
        and coalesce(c.nodeid, c.lion_node_number) is not null
        union select min(s.nodeid) nodeid, integration_id, cast(c.accident_dt as date) accident_dt
        from forms.dbo.wc_accident_f c
        join [FORMS].[dbo].[v_IntersectionStreetNames_Gen] s
        on 
        lower(ltrim(rtrim(c.SRC_ON_STREET))) = lower(ltrim(rtrim(s.street_1))) 
        and
        lower(ltrim(rtrim(c.SRC_cross_STREET))) = lower(ltrim(rtrim(s.street_2)))
        and 
        case WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 1 AND 34 THEN 1
            WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 40 AND 52 THEN 2
            WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 60 AND 94 THEN 3
            WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 100 AND 115 THEN 4
            WHEN C.[SRC_POLICE_PRECINCT] BETWEEN 120 AND 123 THEN 5 end 
        in (left(b7sc_2, 1), left(b7sc_1, 1))
        where year(c.accident_dt) >= 2017
        and accident_dt between '{start}' and '{end}'
        and coalesce(c.nodeid, c.lion_node_number) is null
        group by integration_id, cast(c.accident_dt as date)
        order by nodeid, cast(c.accident_dt as date) desc
        
        """.format(start=start, end=end))
    
    return crashes

In [14]:
crashes = all_crashes()

In [16]:
#crashes